In [1]:
import pandas as pd

In [2]:
Data = pd.read_csv('House_Price_Regression.csv')

In [3]:
Data = Data.loc[:, ['latitude', 'longitude', 'price', 'size_in_m_2']]

Data.head()

,latitude,longitude,price,size_in_m_2
0,25.113208,55.138932,2700000,100.242337
1,25.106809,55.151201,2850000,146.972546
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821


In [4]:
from sklearn.utils import resample

import numpy as np

import math

In [5]:
#Configuracion inicial aleatoria de los clusters:

n=len(Data)

k=4

elementos_clusterizados = []

In [7]:
m = resample(range(0, n), n_samples=math.floor(n/k) , replace=False, random_state=123)

Cluster_1 = Data.loc[m,:]

elementos_clusterizados.append(m)

###################################################################################


if k >= 2:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_2 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_2 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 2 :

    pass

###################################################################################

if k >= 3:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_3 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_3 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 3 :

    pass

###################################################################################

if k >= 4:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_4 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_4 = Data.loc[m ,:]

        elementos_clusterizados.append(m)

elif k < 4 :

    pass

In [8]:
Cluster_1

,latitude,longitude,price,size_in_m_2
382,25.196489,55.272126,15800000,488.019459
732,25.107984,55.244923,1700000,138.704179
1888,25.071504,55.128579,1300000,171.220229
679,25.054336,55.203423,999000,116.035847
1004,25.087251,55.145574,2990000,162.208638
...,...,...,...,...
169,25.075017,55.137997,1990000,144.557068
947,25.195483,55.271797,1483888,56.763733
1426,25.084336,55.149250,2399950,140.190627
1853,25.182869,55.282704,920000,72.278534


In [9]:
Cluster_2

,latitude,longitude,price,size_in_m_2
1041,25.026191,55.156948,350000,35.117334
1064,25.027786,55.156695,399000,35.303140
810,25.088533,55.171838,890000,70.048862
1440,25.060989,55.217377,349000,37.161200
603,25.022371,55.254756,550000,58.993405
...,...,...,...,...
176,25.087251,55.145574,1600000,90.766231
1328,25.051384,55.212100,799000,94.389448
1044,25.077213,55.136939,2395000,174.193125
1084,25.176965,55.311826,1439112,104.887487


In [10]:
Cluster_3

,latitude,longitude,price,size_in_m_2
360,25.086376,55.147360,1890000,116.593265
539,25.068729,55.139815,820000,59.457920
1070,25.233787,55.294217,1359000,91.973970
298,25.099132,55.243982,1021888,68.934026
1218,25.086886,55.173227,2000000,156.077040
...,...,...,...,...
884,25.204058,55.263127,4600000,248.794234
73,25.193310,55.280919,2990000,176.794409
1371,25.048660,55.209550,1190500,146.322225
1315,25.194282,55.271884,2900000,144.464165


In [11]:
Cluster_4

,latitude,longitude,price,size_in_m_2
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821
10,25.198796,55.271342,3550000,178.187954
14,25.132445,55.152216,2089999,97.641053
...,...,...,...,...
1892,24.865992,55.137958,1175000,301.656041
1896,25.196489,55.272126,18040888,488.019459
1900,25.176892,55.310712,1500000,100.985561
1902,25.206500,55.345056,2900000,179.302790


In [13]:
# Calculo de centroides iniciales


centroide_1 = Cluster_1.mean()
centroide_2 = Cluster_2.mean()
centroide_3 = Cluster_3.mean()
centroide_4 = Cluster_4.mean()

In [18]:
# Calculo de distancias Euclideas para la observacion x_1

x_1 = Data.iloc[0,:]

distancia_1 = sum((x_1 - centroide_1)**2)
distancia_2 = sum((x_1 - centroide_2)**2)
distancia_3 = sum((x_1 - centroide_3)**2)
distancia_4 = sum((x_1 - centroide_4)**2)

In [24]:
df_distancias = pd.DataFrame({'Distancias' : [distancia_1 , distancia_2 , distancia_3, distancia_4], 'Cluster': [1,2,3,4]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

df_distancias_sort

In [32]:
#Cluster al que se asigna x_1

j_star = df_distancias_sort.iloc[0]['Cluster']

j_star

3.0

In [73]:
def Cluster(j):

    if j==1:

        cluster = Cluster_1

    elif j==2:

        cluster = Cluster_2

 
    return(cluster)

In [74]:
Cluster(2)

,latitude,longitude,price,size_in_m_2
1041,25.026191,55.156948,350000,35.117334
1064,25.027786,55.156695,399000,35.303140
810,25.088533,55.171838,890000,70.048862
1440,25.060989,55.217377,349000,37.161200
603,25.022371,55.254756,550000,58.993405
...,...,...,...,...
176,25.087251,55.145574,1600000,90.766231
1328,25.051384,55.212100,799000,94.389448
1044,25.077213,55.136939,2395000,174.193125
1084,25.176965,55.311826,1439112,104.887487


In [43]:
# si sum(Cluster_j.index == 1) !=  0 --> x_1 esta en el cluster j


# si x_1 esta en  C1 y se le ha asignado a C1 , es decir,  Cluster(j_star)=Cluster1 --> pasamos a x_2

if sum(Cluster_1.index == 1) !=  0  &  j_star == 1 :

# Calculo de distancias de x_2 respecto los centroides:

x_2 = Data.iloc[1,:]

distancia_1 = sum((x_2 - centroide_1)**2)
distancia_2 = sum((x_2 - centroide_2)**2)
distancia_3 = sum((x_2 - centroide_3)**2)
distancia_4 = sum((x_2 - centroide_4)**2)

# Calculamos el cluster optimo para x_1 con la nueva configuracion de clusters:

df_distancias = pd.DataFrame({'Distancias' : [distancia_1 , distancia_2 , distancia_3, distancia_4], 'Cluster': [1,2,3,4]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

j_star = df_distancias_sort.iloc[0]['Cluster']  # indice del cluster optimo para x_2




# si x_1 esta en  C1 y se le ha asignado a otro cluster Cluster(j_star) --> hay que asignarle a  Cluster(j_star) y recalcular centroides

if sum(Cluster_1.index == 1) !=  0  &  j_star != 1 :

#Añadimos x_1 al cluster 3:

    Cluster(j_star) = pd.concat( [  Cluster(j_star) , Cluster_1.loc[1-1,:].to_frame().T ]  )  # la observacion 1 es la 0=1-1 en python

# Eliminamos x_1 del cluster 1:

    Cluster_1.drop(1-1)  # la observacion 1 es la 0=1-1 en python

# Recalculo de centroides para la nueva configuracion de clusters

     centroide_1 = Cluster_1.mean()
     centroide_2 = Cluster_2.mean()
     centroide_3 = Cluster_3.mean()
     centroide_4 = Cluster_4.mean()

# Recalculo de distancias de x_1 respecto los centroides:

x_1 = Data.iloc[0,:]

distancia_1 = sum((x_1 - centroide_1)**2)
distancia_2 = sum((x_1 - centroide_2)**2)
distancia_3 = sum((x_1 - centroide_3)**2)
distancia_4 = sum((x_1 - centroide_4)**2)

# Calculamos el cluster optimo para x_1 con la nueva configuracion de clusters:

df_distancias = pd.DataFrame({'Distancias' : [distancia_1 , distancia_2 , distancia_3, distancia_4], 'Cluster': [1,2,3,4]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

j_star = df_distancias_sort.iloc[0]['Cluster']  # indice del nuevo cluster optimo para x_1



#Repetir lo anterior con C2, C3 y C4

0

In [21]:
#Configuracion inicial aleatoria de los clusters:

n=len(Data)

k=4

elementos_clusterizados = []

m = resample(range(0, n), n_samples=math.floor(n/k) , replace=False, random_state=123)

Cluster_0 = Data.loc[m,:]

elementos_clusterizados.append(m)


###################################################################################


if k >= 2:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_1 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_1 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 2 :

    pass

###################################################################################

if k >= 3:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_2 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_2 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 3 :

    pass

###################################################################################

if k >= 4:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_3 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_3 = Data.loc[m ,:]

        elementos_clusterizados.append(m)

elif k < 4 :

    pass

In [22]:
lista_clusters = [None] * 4
lista_clusters

[None, None, None, None]

In [23]:
lista_clusters[0] = Cluster_0
lista_clusters[1] = Cluster_1
lista_clusters[2] = Cluster_2
lista_clusters[3] = Cluster_3

In [39]:
# Calculo de centroides iniciales

centroide_0 = lista_clusters[0].mean()
centroide_1 = lista_clusters[1].mean()
centroide_2 = lista_clusters[2].mean()
centroide_3 = lista_clusters[3].mean()

# Calculo de distancias Euclideas para la observacion x_1

x_1 = Data.iloc[0,:]

distancia_1 = sum((x_1 - centroide_0)**2)
distancia_2 = sum((x_1 - centroide_1)**2)
distancia_3 = sum((x_1 - centroide_2)**2)
distancia_4 = sum((x_1 - centroide_3)**2)

# Calculo de cluster optimo para x_1

df_distancias = pd.DataFrame({'Distancias' : [distancia_1 , distancia_2 , distancia_3, distancia_4], 'Cluster': [0,1,2,3]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

j_star = df_distancias_sort.iloc[0]['Cluster']

j_star = int(j_star)

In [40]:
j_star

2

In [41]:
i = 1

while i <= n :

    for j in range(0, k) :


        # Si x_i no esta en el cluster j --> pasamos a analizar otro cluster j

        if sum(lista_clusters[j].index == i-1) ==  0 :

            pass


        # Si x_i esta en el cluster j y es el cluster optimo de x_i --> pasamos a x_i+1

        if ( sum(lista_clusters[j].index == i-1) != 0 )  & ( j_star == j ) :

            # Actualizamos i a i+1

            i = i + 1

            # calculamos las distancias entre x_i+1 y los centroides

            x_i = Data.iloc[ i ,:] # x_i+1

            distancia_1 = sum((x_i - centroide_1)**2)
            distancia_2 = sum((x_i - centroide_2)**2)
            distancia_3 = sum((x_i - centroide_3)**2)
            distancia_4 = sum((x_i - centroide_4)**2)


            # Calculo de cluster optimo para x_1

            df_distancias = pd.DataFrame({'Distancias' : [distancia_1 , distancia_2 , distancia_3, distancia_4], 'Cluster': [1,2,3,4]})

            df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

            j_star = df_distancias_sort.iloc[0]['Cluster']



        # Si x_i esta en el cluster j pero ese no es el cluster optimo de x_i 

        if ( sum(lista_clusters[j].index == i-1) != 0 )  & ( j_star != j ) : 

            ## Actualizamos los clusters:

            #Añadimos x_i al cluster j_star :

            lista_clusters[j_star] = pd.concat( [ lista_clusters[j_star] , lista_clusters[j].loc[i-1,:].to_frame().T ]  )  

            
            # Eliminamos x_1 del cluster j :

            lista_clusters[j] = lista_clusters[j].drop(i-1)   


            # Recalculo de centroides para la nueva configuracion de clusters

            centroide_0 = lista_clusters[0].mean()
            centroide_1 = lista_clusters[1].mean()
            centroide_2 = lista_clusters[2].mean()
            centroide_3 = lista_clusters[3].mean()

            # Recalculo de distancias de x_i respecto los centroides:

            x_i = Data.iloc[i-1,:]

            distancia_0 = sum((x_1 - centroide_1)**2)
            distancia_1 = sum((x_1 - centroide_2)**2)
            distancia_2 = sum((x_1 - centroide_3)**2)
            distancia_3 = sum((x_1 - centroide_4)**2)


IndexError: list index out of range

In [18]:
lista_clusters[j_star] = pd.concat( [ lista_clusters[j_star] , lista_clusters[j].loc[i-1,:].to_frame().T ]  )  


1

In [38]:
int(j_star)

2

In [32]:
lista_clusters

[       latitude  longitude     price  size_in_m_2
 382   25.196489  55.272126  15800000   488.019459
 732   25.107984  55.244923   1700000   138.704179
 1888  25.071504  55.128579   1300000   171.220229
 679   25.054336  55.203423    999000   116.035847
 1004  25.087251  55.145574   2990000   162.208638
 ...         ...        ...       ...          ...
 169   25.075017  55.137997   1990000   144.557068
 947   25.195483  55.271797   1483888    56.763733
 1426  25.084336  55.149250   2399950   140.190627
 1853  25.182869  55.282704    920000    72.278534
 721   25.100040  55.175161   1020000   121.424221
 
 [476 rows x 4 columns],
        latitude  longitude    price  size_in_m_2
 1041  25.026191  55.156948   350000    35.117334
 1064  25.027786  55.156695   399000    35.303140
 810   25.088533  55.171838   890000    70.048862
 1440  25.060989  55.217377   349000    37.161200
 603   25.022371  55.254756   550000    58.993405
 ...         ...        ...      ...          ...
 176   25.0

In [30]:
pd.concat( [ lista_clusters[j_star] , lista_clusters[j].loc[i-1,:].to_frame().T ]  )  

TypeError: list indices must be integers or slices, not numpy.float64

In [20]:
j_star

3.0

In [19]:
for j in range(0, k) :

    print(sum(lista_clusters[j].index == i-1))

1
0
0
0


In [13]:
sum(lista_clusters[1].index == i-1)

0

In [169]:
lista_clusters = [None] * 4

In [174]:
lista[2] = Data

In [179]:
lista[2]

,latitude,longitude,price,size_in_m_2
0,25.113208,55.138932,2700000,100.242337
1,25.106809,55.151201,2850000,146.972546
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821
...,...,...,...,...
1900,25.176892,55.310712,1500000,100.985561
1901,25.166145,55.276684,1230000,70.606280
1902,25.206500,55.345056,2900000,179.302790
1903,25.073858,55.229844,675000,68.748220


In [155]:
lista = []

In [159]:
lista.append(Data)

TypeError: list.append() takes exactly one argument (2 given)

In [168]:
lista[3]= 'a'

IndexError: list assignment index out of range

In [161]:
lista.append(Data.iloc[2:10, ] )

In [163]:
lista[0]

,latitude,longitude,price,size_in_m_2
0,25.113208,55.138932,2700000,100.242337
1,25.106809,55.151201,2850000,146.972546
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821
...,...,...,...,...
1900,25.176892,55.310712,1500000,100.985561
1901,25.166145,55.276684,1230000,70.606280
1902,25.206500,55.345056,2900000,179.302790
1903,25.073858,55.229844,675000,68.748220


In [164]:
lista[1]

,latitude,longitude,price,size_in_m_2
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821
5,25.114275,55.139764,3119900,94.296545
6,25.114275,55.139764,8503600,191.565986
7,25.114275,55.139764,3119900,94.296545
8,25.106668,55.149275,2100000,203.085958
9,25.194935,55.282665,2690000,141.305463
